In [ ]:
#주제변경
#올리브영 웹사이트에 접속하여 립메이크업 제품 카테고리에서 립제품 가격 분석 등 여러 시각화 시도해보기

import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100020006&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&gateCd=Drawer&trackingCd=Cat100000100020006_MID"

res = requests.get(url)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

olive_links = soup.select("#Contents > ul") #olive_links 변수 선택자(#Contents > ul)를 수정하여 ul 요소들을 가져옴

results = []
for link in olive_links:
    n = link.text.strip()
    results.append(n)
print(results)

brands = []
for v in olive_links:
    li_tags = v.select("li") #li_tags 변수를 추가하여 각 ul 요소에 속한 li 태그들을 선택
    for li in li_tags: #내부 for 루프를 추가하여 li_tags에서 각 li 요소의 내용을 가져와 brands 배열
        if li.select_one("div > div > a > span"): # 맨 처음 항은 null이므로 if문
            brand = li.select_one("div > div > a > span").text
            brands.append(brand)
print(brands)

names = []
for v in olive_links:
    li_tags = v.select("li") #li_tags 변수를 추가하여 각 ul 요소에 속한 li 태그들을 선택
    for li in li_tags: #내부 for 루프를 추가하여 li_tags에서 각 li 요소의 내용을 가져와 names 배열
        if li.select_one("div > div > a > p"): # 맨 처음 항은 null이므로 if문
            name = li.select_one("div > div > a > p").text
            names.append(name)
print(names)

prices = []
for v in olive_links:
    li_tags = v.select("li") #li_tags 변수를 추가하여 각 ul 요소에 속한 li 태그들을 선택
    for li in li_tags: #내부 for 루프를 추가하여 li_tags에서 각 li 요소의 내용을 가져와 prices 배열
        if li.select_one("div > p.prd_price > span.tx_cur > span"): # 맨 처음 항은 null이므로 if문
            price = li.select_one("div > p.prd_price > span.tx_cur > span").text
            prices.append(price)
print(prices)


# 추출한 데이터를 이용하여 DataFrame 생성
data = {"브랜드": brands, "상품명": names, "가격": prices}
df = pd.DataFrame(data)

# 가격을 기준으로 데이터프레임을 오름차순으로 정렬
df_sorted = df.sort_values(by="가격")

# 정렬된 데이터프레임의 인덱스 재설정
df_sorted = df_sorted.reset_index(drop=True)

# 정렬된 데이터프레임 출력
print(df_sorted)


# 가격 컬럼의 데이터 타입을 숫자형으로 변환
df_sorted['가격'] = df_sorted['가격'].str.replace(',', '').astype(int)


# 가격이 10,000원 이하인 상품 출력
print("\n<<상품가격이 10000원 이하인 상품>>\n")
filtered_df = df_sorted[df_sorted['가격'] <= 10000]
print(filtered_df)


# 가격 컬럼의 평균, 최댓값, 최솟값 출력

print("\n<<평균,최댓값,최솟값>>\n")

mean_price = df_sorted['가격'].mean()

def remove_decimal(number):#소수점 빼고 정수로 출력
    return int(number)

#평균가격 출력
print("평균 가격:", remove_decimal(mean_price))

#최댓값,최솟값
max_price = df_sorted['가격'].idxmax()  # 최댓값에 해당하는 행의 인덱스 가져오기
min_price = df_sorted['가격'].idxmin()  # 최솟값에 해당하는 행의 인덱스 가져오기


max_product_name = df_sorted.loc[max_price, '상품명']
max_product_price = df_sorted.loc[max_price, '가격']
min_product_name = df_sorted.loc[min_price, '상품명']
min_product_price = df_sorted.loc[min_price, '가격']

print("\n최댓값 상품명:", max_product_name)
print("최댓값 가격:", max_product_price)
print("최솟값 상품명:", min_product_name)
print("최솟값 가격:", min_product_price)

#상품명 검색하기 기능 추가
print("\n<<상품명 검색하기>>")
keyword = input("검색할 상품명을 입력하세요: ")
matched_df = df_sorted[df_sorted['상품명'].str.contains(keyword)]
print(matched_df)


# 한글 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'  # 원하는 한글 폰트로 변경하세요

# 브랜드별 상품 수 시각화
print("\n<브랜드별 상품 수>>")
brand_counts = df_sorted['브랜드'].value_counts()
plt.bar(brand_counts.index, brand_counts.values)
plt.xlabel('브랜드')
plt.ylabel('상품 수')
plt.title('브랜드별 상품 수')
plt.xticks(rotation=45)
plt.show()
#원그래프이용해보기
brand_counts = df_sorted['브랜드'].value_counts()
plt.pie(brand_counts.values, labels=brand_counts.index, autopct='%1.1f%%')
plt.title('브랜드별 상품 비율')
plt.show()


#제품 가격 분포도 확인하고 시각화
print("\n<<제품가격분포>>")
plt.hist(df_sorted['가격'], bins=20)
plt.xlabel('가격')
plt.ylabel('빈도')
plt.title('제품 가격 분포')
plt.show()
#boxplot이용해보기
plt.boxplot(df_sorted['가격'])
plt.ylabel('가격')
plt.title('가격 분포 상자 그림')
plt.show()


#브랜드별 평균 가격 비교
print("\n<<브랜드별 평균 가격 비교>>")
brand_mean_price = df_sorted.groupby('브랜드')['가격'].mean().sort_values(ascending=False)
plt.bar(brand_mean_price.index, brand_mean_price.values)
plt.xlabel('브랜드')
plt.ylabel('평균 가격')
plt.title('브랜드별 평균 가격 비교')
plt.xticks(rotation=45)
plt.show()

# 상품명 길이와 가격 간의 상관관계 히트맵 시각화
# 이를 통해 상품명의 길이가 가격에 어떤 영향을 미치는지, 또는 두 변수 간의 상관성을 확인
# 시각화를 통해 상품명 길이와 가격 간의 상관관계를 확인해 상품명의 길이가 가격에 대한 정보를 제공하는데 도움
# EX 상품명이 길 수록 가격이 높아지는 경향이 있다면, 상품명의 길이가 가격 결정에 영향을 미친다고 볼 수 있음
correlation_matrix = df_sorted[['상품명', '가격']].apply(lambda x: x.astype(str).str.len())
plt.imshow(correlation_matrix, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.title('상품명 길이와 가격의 상관관계')
plt.show()


#가격대별 상품수 시각화
#가격대(예: 0-10000원, 10000-20000원 등)별로 상품 수를 나타내는 막대 그래프를 그리기
#막대그래프를 통해 가격대별로 어떤 상품이 많이 판매되고 있는지 확인가능

# 가격대 범위와 라벨 설정
price_ranges = [0, 10000, 20000, 30000, 40000]
labels = ['0-10000', '10000-20000', '20000-30000', '30000-40000']

# 가격대 컬럼 추가
df_sorted['가격대'] = pd.cut(df_sorted['가격'], price_ranges, labels=labels)

# 가격대별 상품 수 시각화
price_counts = df_sorted['가격대'].value_counts()
plt.bar(price_counts.index, price_counts.values)
plt.xlabel('가격대')
plt.ylabel('상품 수')
plt.title('가격대별 상품 수')
plt.xticks(rotation=45)
plt.show()


# 가격대별 제품명 출력
price_product_names = df_sorted.groupby('가격대')['상품명'].apply(list)
print("\n<<가격대별 제품명>>")
for price_range, product_names in price_product_names.items():
    print(f"가격대: {price_range}")
    for product_name in product_names:
        print(f"- {product_name}")
    print()

#제품명에 가장 많이 쓰인 키워드 빈도수 
#이를 통해 소비자가 선호하는 제품명에 들어가는 키워드를 분석하여 마케팅 분야에 활용가능 하다.
import matplotlib.pyplot as plt
from collections import Counter

# 제품명에서 키워드 추출
keywords = []
for name in df_sorted['상품명']:
    words = name.split()
    keywords.extend(words)

# 키워드 빈도수 계산
keyword_counter = Counter(keywords)

# 가장 많이 사용된 상위 5개 키워드 추출
top_keywords = keyword_counter.most_common(5)
labels, values = zip(*top_keywords)


# 키워드 빈도수 데이터프레임 생성
keyword_df = pd.DataFrame(top_keywords, columns=['키워드', '빈도수'])

# 키워드 빈도수 표로 출력
print("\n<<키워드 빈도수>>")
print(keyword_df)


# 원 그래프로 시각화
plt.pie(values, labels=labels, autopct='%1.1f%%')
plt.title('가장 많이 사용된 키워드')
plt.show()

#키워드 빈도수로 알아낸 결론
#예상과 다르게 '촉촉한','매트' 이런 키워드가 나오지 않고 기본적인 '틴트' 이런 키워드가 나와서 예상했던 결론과 다름

# 브랜드별 가격 분포 비교 시각화
#알 수 있는 결론
#각 브랜드의 제품들이 어떤 가격대에 집중되어 있는지 시각적으로 확인할 수 있음. 이를 통해 브랜드의 가격 정책이나 시장 위치를 파악할 수 있음
plt.figure(figsize=(10, 6))
plt.boxplot([df_sorted[df_sorted['브랜드'] == brand]['가격'] for brand in df_sorted['브랜드'].unique()])
plt.xlabel('브랜드')
plt.ylabel('가격')
plt.title('브랜드별 가격 분포 비교')
plt.xticks(rotation=45)
plt.show()


# 제품명 길이와 가격 간의 상관관계 히트맵 시각화
#각 브랜드의 제품명에서 가장 많이 사용된 키워드를 추출하여 해당 브랜드의 마케팅 전략이나 키워드 선택에 대한 시각을 키울 수 있음
plt.figure(figsize=(10, 6))
correlation_matrix = df_sorted[['상품명', '가격']].apply(lambda x: x.astype(str).str.len())
plt.imshow(correlation_matrix, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.title('상품명 길이와 가격의 상관관계')
plt.xlabel('상품명')
plt.ylabel('가격')
plt.xticks(range(len(df_sorted)), df_sorted['상품명'], rotation=90)
plt.yticks(range(len(df_sorted)), df_sorted['가격'])
plt.show()




